In [3]:
from __future__ import division
import scipy as sp
import numpy as np
from scipy import io
import itertools
import math

In [4]:
data = io.mmread("data/netflix_mm_10000_1000")
data.shape

(10000, 1000)

In [18]:
def RMSE(data, latent):
    userOffset = 0
    movieOffset = data.shape[0]
    
    cx = data.tocoo() 
    err = 0
    for user,movie,rating in itertools.izip(cx.row, cx.col, cx.data):
        vUser = latent[user + userOffset]
        vMovie = latent[movie + movieOffset]
        err += (vUser.dot(vMovie) - rating) ** 2
    return math.sqrt(err / data.nnz)
    #return err
        

In [19]:
def SGD(data, eta = 0.01, lambduh = 0.1, maxit = 2):
    rank = 10
    userOffset = 0
    movieOffset = data.shape[0]
    latent = np.random.rand(sum(data.shape), rank)
    
    it = 0
    innerIt = 0
    cx = data.tocoo() 
    print "Initial RMSE %f" % (RMSE(data, latent))
    while it < maxit:
        for user,movie,rating in itertools.izip(cx.row, cx.col, cx.data):
            vMovie = latent[movie + movieOffset]
            vUser = latent[user + userOffset] 
            vUserTmp = vUser.copy()
            
            e = vUser.dot(vMovie) - rating
            c1 = (1 - eta * lambduh)
            
            vUser[:] = c1 * vUser - eta * e * vMovie
            vMovie[:] = c1 * vMovie - eta * e * vUserTmp
            
            innerIt += 1
            if innerIt % 500 == 0:
                print "%d - %f" % (innerIt, RMSE(data, latent))
        it += 1
SGD(data)

Initial RMSE 1.598029
500 - 1.567379
1000 - 1.545201
1500 - 1.536284
2000 - 1.523275
2500 - 1.516004
3000 - 1.504103
3500 - 1.491759
4000 - 1.485647
4500 - 1.473554
5000 - 1.465145
5500 - 1.455905
6000 - 1.453675
6500 - 1.453627
7000 - 1.447225
7500 - 1.437045
8000 - 1.425864
8500 - 1.420867
9000 - 1.416463
9500 - 1.411679
10000 - 1.407431
10500 - 1.401739
11000 - 1.401477
11500 - 1.401655
12000 - 1.398239
12500 - 1.394971
13000 - 1.394058
13500 - 1.389644
14000 - 1.384597
14500 - 1.381057
15000 - 1.376469
15500 - 1.371889
16000 - 1.370647
16500 - 1.368308
17000 - 1.364784
17500 - 1.360288
18000 - 1.359177
18500 - 1.357892
19000 - 1.353014
19500 - 1.354806
20000 - 1.351708
20500 - 1.345955
21000 - 1.346816
21500 - 1.347408
22000 - 1.344614
22500 - 1.341794
23000 - 1.340247
23500 - 1.340765
24000 - 1.341299
24500 - 1.340075
25000 - 1.341438
25500 - 1.340992
26000 - 1.337518
26500 - 1.336458
27000 - 1.331972
27500 - 1.332800
28000 - 1.331503
28500 - 1.331580
29000 - 1.324865
29500 - 1.31

KeyboardInterrupt: 